In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load the data 
os.listdir('../input')
sales_data = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
item_cat = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
sample_submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
test_data = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')


In [ ]:
sales_data.head()

In [ ]:
test_data.head()

In [ ]:
sample_submission.head()

In [ ]:
# Function that define all the EDA we need 
def EDA(df):
    print("HEAD OF THE DATA ")
    print(df.head())
    print("INFO")
    print(df.info())
    print("Describe")
    print(df.describe())
    print("Columns")
    print(df.columns)
    print("Data Types")
    print(df.dtypes)
    print("Missing Values")
    print(df.isnull().sum())
    print("NULL values")
    print(df.isna().sum())
    print("Shape Of Data")
    print(df.shape)

In [ ]:
#Litle bit of exploration of data

print("Sales Data __________________________________")
EDA(sales_data)
print("Test data_____________________________")
EDA(test_data)
print("Item Categories_____________________________")
EDA(item_cat)
print("Items______________________________________")
EDA(items)
print("Shops_______________________________")
EDA(shops)
print("Sample Submission___________________________________")
EDA(sample_submission)



In [ ]:
#we can see that 'date' column in sales_data is an object but if we want to manipulate  it or want to work on it someway then we have convert it on datetime format
sales_data['date'] = pd.to_datetime(sales_data['date'],format = '%d.%m.%Y')

In [ ]:
sales_data.head()

In [ ]:
#now we will create a pivot tabel by going so we get our data in desired form 
#we want get total count value of an item over the whole month for a shop 
# That why we made shop_id and item_id our indices and date_block_num our column 
# the value we want is item_cnt_day and used sum as aggregating function 
dataset = sales_data.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')

In [ ]:
dataset.head()

In [ ]:
# lets reset our indices, so that data should be in way we can easily manipulate
dataset.reset_index(inplace = True)

In [ ]:
# lets check on our pivot table
dataset.head(3)

In [ ]:
# Now we will merge our pivot table with the test_data because we want to keep the data of items we have
# predict
dataset = pd.merge(test_data,dataset,on = ['item_id','shop_id'],how = 'left')

In [ ]:
dataset.head()

In [ ]:
# lets fill all NaN values with 0
dataset.fillna(0,inplace = True)
# lets check our data now 
dataset.head()

In [ ]:
# we will drop shop_id and item_id because we do not need them
# we are teaching our model how to generate the next sequence 
dataset.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
dataset.head()

In [ ]:
# X we will keep all columns execpt the last one 
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
# the last column is our label
y_train = dataset.values[:,-1:]

# for test we keep all the columns execpt the first one
X_test = np.expand_dims(dataset.values[:,1:],axis = 2)

# lets have a look on the shape 
print(X_train.shape,y_train.shape,X_test.shape)


In [ ]:
# importing libraries required for our model
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
# DEFINE our model 
model_LSTM = Sequential()
model_LSTM.add(LSTM(units =70,input_shape = (X_train.shape[1], X_train.shape[2])))
model_LSTM.add(Dropout(0.4))
model_LSTM.add(Dense(1))

model_LSTM.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
model_LSTM.summary()


In [ ]:
history_lstm = model_LSTM.fit(X_train,y_train,batch_size = 4000,epochs = 12)

In [ ]:
# Plot the loss curves for training
plt.plot(history_lstm.history['loss'], color='b', label="Training loss")
plt.legend(loc='best', shadow=True)

### You can now import the predictions to the submission file 